In [1]:
import argparse
from SoccerNet.utils import getListGames
import os
import json
from tqdm import tqdm
import numpy as np
from SoccerNet.Evaluation.ActionSpotting import evaluate

In [9]:
parser = argparse.ArgumentParser()
# parser.add_argument("--anno_file",default='')
parser.add_argument("--source_test_path",default='/youtu_pedestrian_detection/chengzhilin/program/soccernet/models/MVIT_FULL/outputs_test')#输入test文件夹 
parser.add_argument("--output_test_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/output_full')  #筛选后test文件夹

parser.add_argument("--source_challenge_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/challenge_224_all')#输入challenge文件夹 
parser.add_argument("--output_challenge_path", default="/youtu_pedestrian_detection/zhuhe/soccernet/submit/challenge_224_all_44")  #输出challenge文件夹
parser.add_argument('--window_start', type=int,   default=5,     help='Size of the chunk (in seconds)' )
parser.add_argument('--window_end', type=int,   default=50,     help='Size of the chunk (in seconds)' )
parser.add_argument('--window_gap',  type=int,   default=5,     help='Size of the chunk (in seconds)' )

args=parser.parse_args([])

In [10]:
os.makedirs(args.output_challenge_path,exist_ok='True')

In [11]:
def process_window(window):
    action_dic={}
    res=[]
    for pre in window:
        if pre['label'] not in action_dic:
            action_dic[pre['label']]=pre
        else:
            if float(pre['confidence']) > float(action_dic[pre['label']]['confidence']):
                action_dic[pre['label']]=pre
    for label in action_dic.keys():
        res.append(action_dic[label])
    return res

In [12]:
total_results=[]
window_size=args.window_start
while window_size<=args.window_end:
    print(window_size)

    for cata in getListGames(split="test"):

        file=os.path.join(args.source_test_path,cata,'results_spotting.json')
        with open(file,'r') as f:
            data=json.load(f)
        pres=data['predictions']
        pres.sort(key=lambda x:(int(x['half']),int(x['position'])))

        json_data = {} 
        total_pre={}
        json_data["UrlLocal"]=data['UrlLocal']
        json_data["predictions"]=[]

        last_half=1
        start,end=0,window_size
        window=[]
        for cur_pre,pre in enumerate(pres):


            if start<= int(pre['position'])/1000.0 <end:
                window.append(pre)
            else:
                json_data["predictions"]+=process_window(window)
                window=[pre]
                # while end<int(pre['position']):
                start+=window_size
                end+=window_size

                if last_half==1 and int(pre['half'])==2:
                    last_half=2
                    start,end=0,window_size

        json_data["predictions"]+=process_window(window)

        os.makedirs(os.path.join(args.output_test_path,cata),exist_ok='True')
        with open(os.path.join(args.output_test_path,cata,'results_spotting.json'),'w') as f:
            json.dump(json_data,f,indent=2)
    
    PATH_DATASET='/youtu_pedestrian_detection/junweil/action_datasets/soccernet'
    PATH_PREDICTIONS=args.output_test_path


    from SoccerNet.Evaluation.ActionSpotting import evaluate
    results = evaluate(SoccerNet_path=PATH_DATASET, Predictions_path=PATH_PREDICTIONS,
                       split="test", version=2, prediction_file="results_spotting.json", metric="tight")

    a_map,a_mAP_visible,a_mAP_unshown=results["a_mAP"],results["a_mAP_visible"],results["a_mAP_unshown"]
    total_results.append([window_size,a_map,a_mAP_visible,a_mAP_unshown])
    
    
    window_size+=args.window_gap

5


100%|██████████| 5/5 [00:15<00:00,  3.10s/it]


10


100%|██████████| 5/5 [00:14<00:00,  2.92s/it]


15


...

100%|██████████| 5/5 [00:11<00:00,  2.38s/it]


45


100%|██████████| 5/5 [00:11<00:00,  2.33s/it]


50


100%|██████████| 5/5 [00:10<00:00,  2.19s/it]


In [6]:
window_size

55

In [7]:
total_results.sort(key=lambda x:-x[1])
window_size=total_results[0][0]

In [8]:
total_results

[[5, 0.4316695379520418, 0.4865320724428931, 0.20420540461736453],
 [10, 0.4316695379520418, 0.4865320724428931, 0.20420540461736453],
 [15, 0.43155757228072034, 0.48652779563978565, 0.20420641228212888],
 [20, 0.43101887625793367, 0.4863902062807423, 0.2024850641793199],
 [25, 0.4302990448934521, 0.4856189902829648, 0.20072032148763908],
 [30, 0.42859774746924023, 0.48518401535011146, 0.20012038474497285],
 [35, 0.4257600044493174, 0.4818685940276057, 0.195511294114514],
 [40, 0.4237343944576624, 0.479343856295554, 0.19303794287031015],
 [45, 0.42234392115825115, 0.47806430555255935, 0.1906512938071285],
 [50, 0.4199582269886099, 0.47641961584532116, 0.18818654403318574]]

In [112]:
window_size

10.0

In [113]:
total_results

[[10.0, 0.3635886168702596, 0.4452361951226681, 0.13416065650464098],
 [9.0, 0.3628183949734276, 0.4408861857435958, 0.13129837213006507],
 [8.0, 0.3623251507985085, 0.4394945123104102, 0.13513755799699895],
 [9.5, 0.36051810512867954, 0.43839204361662265, 0.13315908195648835],
 [8.5, 0.3597629987587144, 0.440094394295939, 0.13528863884583778],
 [7.0, 0.3574157910817314, 0.4357392284301309, 0.13575448732562678],
 [6.5, 0.3571480587524357, 0.43471564446898003, 0.13501414585936694],
 [7.5, 0.35633529300113514, 0.43441008626314975, 0.13633516850227909],
 [5.5, 0.35045527138076066, 0.4247491026570801, 0.13260608574209778],
 [6.0, 0.3503767726499682, 0.42360994858021006, 0.1333123736889443],
 [5.0, 0.3486040402767273, 0.4253135064499454, 0.1337106279009972],
 [4.0, 0.34435912502347854, 0.41643962807336105, 0.1324311198498365],
 [4.5, 0.3424705960086811, 0.41638542665409883, 0.1315793889648043],
 [3.5, 0.3370436435125208, 0.40952792182902686, 0.13061871362556607],
 [3.0, 0.32788232376722665,

In [13]:
window_size

26.5

In [12]:
for cata in tqdm(getListGames(split="challenge")):
    
    file=os.path.join(args.source_challenge_path,cata,'results_spotting.json')
    with open(file,'r') as f:
        data=json.load(f)
    pres=data['predictions']
    pres.sort(key=lambda x:(int(x['half']),int(x['position'])))
    
    json_data = {} 
    total_pre={}
    json_data["UrlLocal"]=data['UrlLocal']
    json_data["predictions"]=[]
    
    last_half=1
    start,end=0,window_size
    window=[]
    for cur_pre,pre in enumerate(pres):

        
        if start<= int(pre['position'])/1000.0 <end:
            window.append(pre)
        else:
            json_data["predictions"]+=process_window(window)
            window=[pre]
            start+=window_size
            end+=window_size
            
            if last_half==1 and int(pre['half'])==2:
                last_half=2
                start,end=0,window_size
         
    json_data["predictions"]+=process_window(window)
    
    os.makedirs(os.path.join(args.output_challenge_path,cata),exist_ok='True')
    with open(os.path.join(args.output_challenge_path,cata,'results_spotting.json'),'w') as f:
        json.dump(json_data,f,indent=2)
    
    


100%|██████████| 50/50 [00:08<00:00,  5.74it/s]
